In [22]:
import pandas as pd

In [25]:
df = pd.read_csv('onlinefoods.csv')

In [24]:
df = df.drop(columns=['Unnamed: 12'])

In [21]:
df_new = df.select_dtypes(include = ['object']).copy()
nRows, nCols = df_new.shape
for myIndex in range(0, nCols):
    headerName = df_new.columns[myIndex]
    df_new[headerName] = df_new[headerName].astype("category")
    df_new[headerName] = df_new[headerName].cat.codes
    df[headerName] = df_new[headerName]
df_numeric = df
df_numeric.head()

,Age,Gender,Marital Status,Occupation,Monthly Income,Educational Qualifications,Family size,latitude,longitude,Pin code,Output,Feedback
0,20,0,2,3,4,2,4,12.9766,77.5993,560001,1,1
1,24,0,2,3,2,0,3,12.9770,77.5773,560009,1,1
2,22,1,2,3,2,2,3,12.9551,77.6593,560017,1,0
3,22,0,2,3,4,0,6,12.9473,77.5616,560019,1,1
4,22,1,2,3,2,2,4,12.9850,77.5533,560010,1,1


In [16]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
numeric = ['Age', 'Family size', 'latitude', 'longitude', 'Pin code']
df_scaled = df.copy()
standardscaler = StandardScaler()
df_scaled[numeric] = standardscaler.fit_transform(df[numeric])

In [17]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score

df_svm = df

categorical = ['Gender', 'Marital Status', 'Occupation', 'Monthly Income',
               'Educational Qualifications', 'Feedback']

label_encoders = {}
for column in categorical:
    le = LabelEncoder()
    df_svm[column] = le.fit_transform(df_svm[column])
    label_encoders[column] = le

X = df_svm.drop(columns=['Output'])
y = LabelEncoder().fit_transform(df_svm['Output'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y,
                                                     test_size = 0.2,
                                                     random_state = 42)

svm_model = SVC(kernel = 'linear', C = 1.0, random_state = 42)

pipeline = Pipeline([
    ('scaler', scaler),
    ('svm', svm_model)
])

param_grid = {
    'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'svm__C': [0.01, 0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto'],
    'svm__degree': [2, 3, 4],
}
grid_search = GridSearchCV(estimator = pipeline, param_grid = param_grid, cv = 5, scoring = 'accuracy', n_jobs = -1, verbose = 1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

import pickle
filename = 'modell.sav'
pickle.dump(best_model,open(filename,'wb'))

Fitting 5 folds for each of 120 candidates, totalling 600 fits
